Aşağıdaki not defterlerini çalıştırmak için, henüz yapmadıysanız, .env dosyasının içine openai anahtarını `OPENAI_API_KEY` olarak ayarlamanız gerekir.


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Şimdi, Gömme İndeksini bir Pandas Dataframe'ine yükleyeceğiz. Gömme İndeksi, `embedding_index_3m.json` adlı bir JSON dosyasında saklanır. Gömme İndeksi, Ekim 2023 sonuna kadar olan YouTube transkriptlerinin gömmelerini içerir.


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Şimdi, sorgu için Embedding Index'te arama yapacak olan `get_videos` adlı bir fonksiyon oluşturacağız. Bu fonksiyon, sorguya en çok benzeyen ilk 5 videoyu döndürecek. Fonksiyonun çalışma şekli şöyle:

1. Öncelikle, Embedding Index'in bir kopyası oluşturulur.
2. Ardından, sorgunun Embedding'i OpenAI Embedding API kullanılarak hesaplanır.
3. Sonra, Embedding Index'te `similarity` adında yeni bir sütun oluşturulur. `similarity` sütunu, sorgu Embedding'i ile her video segmentinin Embedding'i arasındaki kosinüs benzerliğini içerir.
4. Daha sonra, Embedding Index `similarity` sütununa göre filtrelenir. Embedding Index, yalnızca kosinüs benzerliği 0.75 veya daha yüksek olan videoları içerecek şekilde filtrelenir.
5. Son olarak, Embedding Index `similarity` sütununa göre sıralanır ve en iyi 5 video döndürülür.


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Bu fonksiyon çok basittir, sadece arama sorgusunun sonuçlarını yazdırır.


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. Öncelikle, Gömme İndeksi bir Pandas Dataframe’ine yüklenir.
2. Sonrasında, kullanıcıdan bir sorgu girmesi istenir.
3. Ardından, `get_videos` fonksiyonu çağrılarak Gömme İndeksi’nde sorgu aranır.
4. Son olarak, `display_results` fonksiyonu çağrılarak sonuçlar kullanıcıya gösterilir.
5. Kullanıcıdan tekrar bir sorgu girmesi istenir. Bu süreç, kullanıcı `exit` yazana kadar devam eder.

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.tr.png)

Bir sorgu girmeniz istenecek. Bir sorgu girip enter’a basın. Uygulama, sorgunuzla ilgili videoların bir listesini döndürecektir. Ayrıca, sorunun cevabının videoda bulunduğu yere bir bağlantı da sunulacaktır.

Deneyebileceğiniz bazı sorgular şunlardır:

- Azure Machine Learning nedir?
- Evrişimli sinir ağları nasıl çalışır?
- Sinir ağı nedir?
- Azure Machine Learning ile Jupyter Notebooks kullanabilir miyim?
- ONNX nedir?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**Feragatname**:
Bu belge, AI çeviri hizmeti [Co-op Translator](https://github.com/Azure/co-op-translator) kullanılarak çevrilmiştir. Doğruluk için çaba göstersek de, otomatik çevirilerde hata veya yanlışlıklar bulunabileceğini lütfen unutmayın. Belgenin orijinal diliyle hazırlanmış hali esas alınmalıdır. Kritik bilgiler için profesyonel insan çevirisi önerilir. Bu çevirinin kullanımından doğabilecek yanlış anlama veya yanlış yorumlamalardan sorumlu değiliz.
